In [65]:
#ngboost and modelling libraries
from ngboost import NGBClassifier
from ngboost.distns import Bernoulli
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import roc_auc_score, roc_curve, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from collections import Counter

#data manipulation libraries
import pandas as pd
import numpy as np

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

#import functions and constants
from utils import (register_imputation, preprocess_df, 
                    scaling_values_df, clean_outliers, get_sample_weights, 
                  process_unit_cost, cost_score, generate_y_pred_with_custom_threshold, 
                  calculate_cost_score_with_learning, calculate_learning_cost, process_learning_unit_cost,)
from plot_utils import (correlation_heatmap, visualize_permutation_feature_importances, 
                        plot_target_balance, plot_trace_line, plot_scatter_matrix, 
                        plot_feature_importances, visualize_roc_curve, color_negative_red, )

from constants import cols, cols_with_missing_indicators

## READING DATAFRAME

In [27]:
df = pd.read_csv("./data/cs-training.csv", usecols =cols)

In [28]:
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [29]:
len(df)

150000

## FEATURE ENGINEERING

In [30]:

def preprocess_extra(df):
    """
    Extra cleanup process of some outliers.
    """
    df.loc[
        (df["RevolvingUtilizationOfUnsecuredLines"] > 1),
        "RevolvingUtilizationOfUnsecuredLines",
    ] = 0
    df.loc[(df["DebtRatio"] > 10), "DebtRatio"] = 0
    df.loc[(df["age"] < 18), "age"] = 0

In [31]:
from sklearn.neighbors import LocalOutlierFactor
def clean_outliers(df, flag_filter=False):
    """ 
    Register imputations, identify outliers with LOF and clean them. Also it process an extra function to clean some outliers.

    Args:
        - df (DataFrame Object): dataframe to be processed
        - flag_filter (boolean): Flag that indicates if the process requres an extra cleanup of outliers.

    Returns dataframe without outliers
    """
    df = preprocess_df(df.copy())
    #df = register_imputation(df.copy())
    local_outlier_factor = LocalOutlierFactor(contamination=0.1)
    is_outlier = local_outlier_factor.fit_predict(df[cols[1:]]) == -1
    data_outlier_excluded = df.loc[~is_outlier, :]
    if flag_filter:
        preprocess_extra(data_outlier_excluded)
    return data_outlier_excluded

In [32]:
df = clean_outliers(df.copy(), True)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135000 entries, 0 to 149999
Data columns (total 11 columns):
SeriousDlqin2yrs                        135000 non-null int64
RevolvingUtilizationOfUnsecuredLines    135000 non-null float64
age                                     135000 non-null int64
NumberOfTime30-59DaysPastDueNotWorse    135000 non-null int64
DebtRatio                               135000 non-null float64
MonthlyIncome                           135000 non-null float64
NumberOfOpenCreditLinesAndLoans         135000 non-null int64
NumberOfTimes90DaysLate                 135000 non-null int64
NumberRealEstateLoansOrLines            135000 non-null int64
NumberOfTime60-89DaysPastDueNotWorse    135000 non-null int64
NumberOfDependents                      135000 non-null float64
dtypes: float64(4), int64(7)
memory usage: 12.4 MB


In [34]:
print(df.isnull().sum())

SeriousDlqin2yrs                        0
RevolvingUtilizationOfUnsecuredLines    0
age                                     0
NumberOfTime30-59DaysPastDueNotWorse    0
DebtRatio                               0
MonthlyIncome                           0
NumberOfOpenCreditLinesAndLoans         0
NumberOfTimes90DaysLate                 0
NumberRealEstateLoansOrLines            0
NumberOfTime60-89DaysPastDueNotWorse    0
NumberOfDependents                      0
dtype: int64


In [35]:
df.describe()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
count,135000.000000,135000.000000,135000.000000,135000.000000,135000.000000,135000.000000,135000.000000,135000.000000,135000.000000,135000.000000,135000.000000
mean,0.060726,0.294336,52.143319,0.280696,0.328392,5089.181052,8.098289,0.134000,0.996363,0.117170,0.724215
std,0.238828,0.335301,14.336519,2.515978,0.673520,5747.339847,4.526645,2.472258,1.045003,2.459597,1.084068
min,0.000000,0.000000,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.024932,41.000000,0.000000,0.003842,1251.750000,5.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.132987,52.000000,0.000000,0.214178,4250.000000,7.000000,0.000000,1.000000,0.000000,0.000000
75%,0.000000,0.499763,62.000000,0.000000,0.412272,7200.000000,11.000000,0.000000,2.000000,0.000000,1.000000
max,1.000000,1.000000,99.000000,98.000000,10.000000,151855.000000,57.000000,98.000000,32.000000,98.000000,10.000000


## NGBOOST MODEL

In [36]:
X = df.drop(columns=['SeriousDlqin2yrs']).values
y = df.SeriousDlqin2yrs.values

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
import random
list_of_ids=random.sample(range(0,len(X_train)), int(len(X_train) * 0.2))

In [39]:
df_train = pd.DataFrame(X_train, columns = cols [1:])
df_train

,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,0.074229,52.0,0.0,0.019753,69000.0,10.0,0.0,1.0,0.0,1.0
1,0.677837,29.0,0.0,0.481669,3545.0,7.0,0.0,1.0,0.0,2.0
2,0.113926,59.0,0.0,0.392699,4300.0,12.0,0.0,1.0,0.0,2.0
3,0.939826,55.0,0.0,0.363687,1800.0,9.0,0.0,0.0,0.0,0.0
4,0.949051,47.0,0.0,0.126517,4283.0,3.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
107995,0.010324,58.0,0.0,0.406999,5800.0,11.0,0.0,2.0,0.0,0.0
107996,0.193164,84.0,0.0,0.910072,833.0,11.0,0.0,0.0,0.0,0.0
107997,0.742393,46.0,0.0,0.509249,10000.0,8.0,1.0,2.0,0.0,2.0
107998,0.098779,41.0,2.0,0.117085,4500.0,6.0,0.0,0.0,0.0,2.0


In [41]:
for idx in list_of_ids:
    for col in ['MonthlyIncome', 'NumberRealEstateLoansOrLines', 'NumberOfOpenCreditLinesAndLoans']: #en el segundo lugar estaba dummy_MonthlyIncome
        df_train.at[idx, col] = np.nan

In [42]:
def register_imputation(df):
    """
    Register imputations of certain df
    Args:
        - df (DataFrame): Dataframe to be computed
    Return df with filled values and booleans that indicate if each row was changed
    """
    for c in cols[1:]:
        # df[f"{c}_amputado"] = df[f"{c}"].fillna(0)
        df[f"{c}_imputed"] = df[f"{c}"].fillna(0)
        df[f"dummy_{c}"] = (df[f"{c}"] != df[f"{c}_imputed"]).astype(int)
        del df[f"{c}"]
        df.columns = df.columns.str.replace(f"{c}_imputed", f"{c}")
    return df


In [43]:
df_train = register_imputation(df_train.copy())

In [45]:
df_train.dummy_MonthlyIncome.sum()

21600

In [84]:
len(df_train)

108000

In [85]:
108000 * 0.2

21600.0

In [46]:
X_train = df_train.values

In [56]:
pd.DataFrame(X_test)

,0,1,2,3,4,5,6,7,8,9
0,0.524218,71.0,0.0,0.489049,4565.0,5.0,0.0,2.0,0.0,0.0
1,0.015158,67.0,0.0,0.003222,9000.0,6.0,0.0,0.0,0.0,1.0
2,0.004222,42.0,0.0,0.380055,11250.0,6.0,0.0,2.0,0.0,2.0
3,0.950403,45.0,0.0,0.468553,10000.0,8.0,0.0,3.0,0.0,2.0
4,0.031566,66.0,0.0,0.000000,0.0,9.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
26995,0.546383,30.0,0.0,0.829894,4167.0,6.0,0.0,2.0,0.0,2.0
26996,0.014533,57.0,0.0,0.008213,2556.0,3.0,0.0,0.0,0.0,0.0
26997,0.206702,41.0,0.0,0.307175,8333.0,6.0,0.0,1.0,0.0,2.0
26998,0.054332,40.0,0.0,0.248834,4500.0,6.0,0.0,1.0,0.0,2.0


In [47]:
nm_common =SMOTE(random_state=2019)
X_train_resampled, y_train_resampled = nm_common.fit_sample(X_train, y_train)
print("Resampled dataset shape {}".format(Counter(y_train_resampled)))

Resampled dataset shape Counter({0: 101492, 1: 101492})


In [48]:
ngb_clf = NGBClassifier(Base=DecisionTreeRegressor(ccp_alpha=0.0,
                                         criterion='friedman_mse', max_depth=16,
                                         max_features=None, max_leaf_nodes=None,
                                         min_impurity_decrease=0.0,
                                         min_impurity_split=None,
                                         min_samples_leaf=1,
                                         min_samples_split=2,
                                         min_weight_fraction_leaf=0.0,
                                         presort='deprecated',
                                         random_state=None, splitter='best'),
             col_sample=1.0,
              learning_rate=0.01, minibatch_frac=1.0, n_estimators=100,
              natural_gradient=True, tol=0.0001,
              verbose=True, verbose_eval=0)
ngb_clf.fit(X_train_resampled, y_train_resampled, sample_weight= get_sample_weights(y_train, y_train_resampled))

NGBClassifier(Base=DecisionTreeRegressor(ccp_alpha=0.0,
                                         criterion='friedman_mse', max_depth=16,
                                         max_features=None, max_leaf_nodes=None,
                                         min_impurity_decrease=0.0,
                                         min_impurity_split=None,
                                         min_samples_leaf=1,
                                         min_samples_split=2,
                                         min_weight_fraction_leaf=0.0,
                                         presort='deprecated',
                                         random_state=None, splitter='best'),
              Dist=<class 'ngboost.distns.categorical.k_categorical.<locals>.Categorical'>,
              Score=<class 'ngboost.scores.LogScore'>, col_sample=1.0,
              learning_rate=0.01, minibatch_frac=1.0, n_estimators=100,
              natural_gradient=True,
              random_state=RandomState(MT

In [52]:
cols_with_missing_indicators

['RevolvingUtilizationOfUnsecuredLines',
 'RevolvingUtilizationOfUnsecuredLines_bool_missing',
 'age',
 'age_bool_missing',
 'NumberOfTime30-59DaysPastDueNotWorse',
 'NumberOfTime30-59DaysPastDueNotWorse_bool_missing',
 'DebtRatio',
 'DebtRatio_bool_missing',
 'MonthlyIncome',
 'MonthlyIncome_bool_missing',
 'NumberOfOpenCreditLinesAndLoans',
 'NumberOfOpenCreditLinesAndLoans_bool_missing',
 'NumberOfTimes90DaysLate',
 'NumberOfTimes90DaysLate_bool_missing',
 'NumberRealEstateLoansOrLines',
 'NumberRealEstateLoansOrLines_bool_missing',
 'NumberOfTime60-89DaysPastDueNotWorse',
 'NumberOfTime60-89DaysPastDueNotWorse_bool_missing',
 'NumberOfDependents',
 'NumberOfDependents_bool_missing']

In [58]:
X_test = register_imputation(pd.DataFrame(X_test, columns = cols[1:])).values

In [63]:
df_aux = pd.DataFrame(X_test, columns=cols_with_missing_indicators)
df_aux['predicted'] = generate_y_pred_with_custom_threshold(ngb_clf, X_test, 0.75)
df_aux['real'] = list(y_test)
df_aux['LoanPrincipal'] = df_aux.MonthlyIncome * 2
cost_score(df_aux.LoanPrincipal, df_aux.predicted, df_aux.real)

count_zero 19419
count_one 7581


7325163.380000046

In [64]:
visualize_roc_curve(ngb_clf, X_train, y_train, X_test, y_test)

TRAIN: ROC AUC=0.984
TEST: ROC AUC=0.756


In [68]:
import random
random.randint(10000, 1000000)

561407

In [69]:
len(range(0,100))

100

In [89]:
base0 = DecisionTreeRegressor(criterion="friedman_mse", max_depth=6)
base1 = DecisionTreeRegressor(criterion="friedman_mse", max_depth=8)
base2 = DecisionTreeRegressor(criterion="friedman_mse", max_depth=12)
base3 = DecisionTreeRegressor(criterion="friedman_mse", max_depth=16)

In [90]:
#FIX MEEEEEEEEEEE: REFACTOR PLZZ !!!
df_collector = pd.DataFrame(columns=  ["alpha","hyperparams", "estimators", 
                                       "learning_rate", "max_depth", "threshold", 
                                       "cost", 
                                       "learning_cost",
                                       "train_auc", "test_auc"])
df_collector['hyperparams'] = df_collector['hyperparams'].astype('object')
k = 0
for estimator in [50, 75, 100, 120, 150]:
    for lr in [0.1, 0.01, 0.001]:
        for baset in [base0,base1,base2,base3]:
            ngb_clf = NGBClassifier(Dist=Bernoulli,
                                      verbose=True, Base=baset, n_estimators = estimator, 
                                    learning_rate = lr, verbose_eval = 0
                                     )
            print(ngb_clf)
            ngb_clf.fit(X_train_resampled, y_train_resampled, sample_weight= get_sample_weights(y_train, y_train_resampled))
            index_th = 0
            #en el primer iteraba el threashold desde 0.1 a 0.9 obviamente cuanto mas menor me daba peor.. entonces itero
            #solamente d esde 0.70 a 0.95
            for threshold in list(np.arange(0.7, 0.95, 0.05)):
                for kix in range(0,25):
                    alpha = random.randint(10000, 1000000)
                    df_collector.ix[str(k),'alpha'] = alpha
                    df_collector.ix[str(k),'hyperparams'] = ngb_clf
                    df_collector.ix[str(k),'estimators'] = estimator
                    df_collector.ix[str(k),'learning_rate'] = lr
                    df_collector.ix[str(k),'max_depth'] = baset
                    threshold = round(threshold,2)
                    print("k: "+str(k))
                    df_aux = pd.DataFrame(X_test, columns=cols_with_missing_indicators)
                    df_aux['predicted'] = generate_y_pred_with_custom_threshold(ngb_clf, X_test, threshold)
                    df_aux['proba_predicted'] = [proba[1] for proba in ngb_clf.predict_proba(X_test)]
                    df_aux['real'] = list(y_test)
                    df_aux['LoanPrincipal'] = df_aux.MonthlyIncome * 2
                    print(threshold)
                    print("cost "+str(cost_score(df_aux.LoanPrincipal, df_aux.predicted, df_aux.real)))
                    print("th "+str(threshold))
                    df_collector.ix[str(k),'threshold'] = threshold
                    df_collector.ix[str(k),'cost'] = cost_score(df_aux.LoanPrincipal, df_aux.predicted, df_aux.real)
                    df_collector.ix[str(k),'learning_cost'] = calculate_learning_cost(df_aux.predicted,df_aux.proba_predicted, alpha)
                    # predict probabilities
                    train_probs = ngb_clf.predict_proba(X_train)
                    test_probs = ngb_clf.predict_proba(X_test)
                    # keep probabilities for the positive outcome only
                    train_probs = train_probs[:, 1]
                    test_probs = test_probs[:, 1]
                    # calculate scores
                    train_auc = roc_auc_score(y_train, train_probs)
                    test_auc = roc_auc_score(y_test, test_probs)
                    df_collector.ix[str(k),'train_auc'] = train_auc
                    df_collector.ix[str(k),'test_auc'] = test_auc
                    k+=1
                    print("sumando k")
                    print("---------------------------------")
            del ngb_clf
df_collector['final_cost'] = df_collector['cost'] - df_collector['learning_cost']

NGBClassifier(Base=DecisionTreeRegressor(ccp_alpha=0.0,
                                         criterion='friedman_mse', max_depth=6,
                                         max_features=None, max_leaf_nodes=None,
                                         min_impurity_decrease=0.0,
                                         min_impurity_split=None,
                                         min_samples_leaf=1,
                                         min_samples_split=2,
                                         min_weight_fraction_leaf=0.0,
                                         presort='deprecated',
                                         random_state=None, splitter='best'),
              Dist=<class 'ngboost.distns.categorical.k_categorical.<locals>.Categorical'>,
              Score=<class 'ngboost.scores.LogScore'>, col_sample=1.0,
              learning_rate=0.1, minibatch_frac=1.0, n_estimators=50,
              natural_gradient=True,
              random_state=RandomState(MT199

sumando k
---------------------------------
k: 62
count_zero 8756
count_one 18244
0.6
cost 2490777.959999984
th 0.6
sumando k
---------------------------------
k: 63
count_zero 8756
count_one 18244
0.6
cost 2490777.959999984
th 0.6
sumando k
---------------------------------
k: 64
count_zero 8756
count_one 18244
0.6
cost 2490777.959999984
th 0.6
sumando k
---------------------------------
k: 65
count_zero 8756
count_one 18244
0.6
cost 2490777.959999984
th 0.6
sumando k
---------------------------------
k: 66
count_zero 8756
count_one 18244
0.6
cost 2490777.959999984
th 0.6
sumando k
---------------------------------
k: 67
count_zero 8756
count_one 18244
0.6
cost 2490777.959999984
th 0.6
sumando k
---------------------------------
k: 68
count_zero 8756
count_one 18244
0.6
cost 2490777.959999984
th 0.6
sumando k
---------------------------------
k: 69
count_zero 8756
count_one 18244
0.6
cost 2490777.959999984
th 0.6
sumando k
---------------------------------
k: 70
count_zero 8756
count_

0.65
cost 2422171.4399999892
th 0.65
sumando k
---------------------------------
k: 132
count_zero 7513
count_one 19487
0.65
cost 2422171.4399999892
th 0.65
sumando k
---------------------------------
k: 133
count_zero 7513
count_one 19487
0.65
cost 2422171.4399999892
th 0.65
sumando k
---------------------------------
k: 134
count_zero 7513
count_one 19487
0.65
cost 2422171.4399999892
th 0.65
sumando k
---------------------------------
k: 135
count_zero 7513
count_one 19487
0.65
cost 2422171.4399999892
th 0.65
sumando k
---------------------------------
k: 136
count_zero 7513
count_one 19487
0.65
cost 2422171.4399999892
th 0.65
sumando k
---------------------------------
k: 137
count_zero 7513
count_one 19487
0.65
cost 2422171.4399999892
th 0.65
sumando k
---------------------------------
k: 138
count_zero 7513
count_one 19487
0.65
cost 2422171.4399999892
th 0.65
sumando k
---------------------------------
k: 139
count_zero 7513
count_one 19487
0.65
cost 2422171.4399999892
th 0.65
sum

sumando k
---------------------------------
k: 200
count_zero 6116
count_one 20884
0.7
cost 2384410.179999987
th 0.7
sumando k
---------------------------------
k: 201
count_zero 6116
count_one 20884
0.7
cost 2384410.179999987
th 0.7
sumando k
---------------------------------
k: 202
count_zero 6116
count_one 20884
0.7
cost 2384410.179999987
th 0.7
sumando k
---------------------------------
k: 203
count_zero 6116
count_one 20884
0.7
cost 2384410.179999987
th 0.7
sumando k
---------------------------------
k: 204
count_zero 6116
count_one 20884
0.7
cost 2384410.179999987
th 0.7
sumando k
---------------------------------
k: 205
count_zero 6116
count_one 20884
0.7
cost 2384410.179999987
th 0.7
sumando k
---------------------------------
k: 206
count_zero 6116
count_one 20884
0.7
cost 2384410.179999987
th 0.7
sumando k
---------------------------------
k: 207
count_zero 6116
count_one 20884
0.7
cost 2384410.179999987
th 0.7
sumando k
---------------------------------
k: 208
count_zero 61

0.7
cost 2384410.179999987
th 0.7
sumando k
---------------------------------
k: 271
count_zero 6116
count_one 20884
0.7
cost 2384410.179999987
th 0.7
sumando k
---------------------------------
k: 272
count_zero 6116
count_one 20884
0.7
cost 2384410.179999987
th 0.7
sumando k
---------------------------------
k: 273
count_zero 6116
count_one 20884
0.7
cost 2384410.179999987
th 0.7
sumando k
---------------------------------
k: 274
count_zero 6116
count_one 20884
0.7
cost 2384410.179999987
th 0.7
sumando k
---------------------------------
k: 275
count_zero 6116
count_one 20884
0.7
cost 2384410.179999987
th 0.7
sumando k
---------------------------------
k: 276
count_zero 6116
count_one 20884
0.7
cost 2384410.179999987
th 0.7
sumando k
---------------------------------
k: 277
count_zero 6116
count_one 20884
0.7
cost 2384410.179999987
th 0.7
sumando k
---------------------------------
k: 278
count_zero 6116
count_one 20884
0.7
cost 2384410.179999987
th 0.7
sumando k
--------------------

...

cost 3863729.799999991
th 0.85
sumando k
---------------------------------
k: 5464
count_zero 9607
count_one 17393
0.85
cost 3863729.799999991
th 0.85
sumando k
---------------------------------
k: 5465
count_zero 9607
count_one 17393
0.85
cost 3863729.799999991
th 0.85
sumando k
---------------------------------
k: 5466
count_zero 9607
count_one 17393
0.85
cost 3863729.799999991
th 0.85
sumando k
---------------------------------
k: 5467
count_zero 9607
count_one 17393
0.85
cost 3863729.799999991
th 0.85
sumando k
---------------------------------
k: 5468
count_zero 9607
count_one 17393
0.85
cost 3863729.799999991
th 0.85
sumando k
---------------------------------
k: 5469
count_zero 9607
count_one 17393
0.85
cost 3863729.799999991
th 0.85
sumando k
---------------------------------
k: 5470
count_zero 9607
count_one 17393
0.85
cost 3863729.799999991
th 0.85
sumando k
---------------------------------
k: 5471
count_zero 9607
count_one 17393
0.85
cost 3863729.799999991
th 0.85
sumando k

count_zero 55
count_one 26945
0.9
cost 2530651.999999984
th 0.9
sumando k
---------------------------------
k: 5534
count_zero 55
count_one 26945
0.9
cost 2530651.999999984
th 0.9
sumando k
---------------------------------
k: 5535
count_zero 55
count_one 26945
0.9
cost 2530651.999999984
th 0.9
sumando k
---------------------------------
k: 5536
count_zero 55
count_one 26945
0.9
cost 2530651.999999984
th 0.9
sumando k
---------------------------------
k: 5537
count_zero 55
count_one 26945
0.9
cost 2530651.999999984
th 0.9
sumando k
---------------------------------
k: 5538
count_zero 55
count_one 26945
0.9
cost 2530651.999999984
th 0.9
sumando k
---------------------------------
k: 5539
count_zero 55
count_one 26945
0.9
cost 2530651.999999984
th 0.9
sumando k
---------------------------------
k: 5540
count_zero 55
count_one 26945
0.9
cost 2530651.999999984
th 0.9
sumando k
---------------------------------
k: 5541
count_zero 55
count_one 26945
0.9
cost 2530651.999999984
th 0.9
sumando 

k: 5600
count_zero 0
count_one 27000
0.6
cost 2471674.099999983
th 0.6
sumando k
---------------------------------
k: 5601
count_zero 0
count_one 27000
0.6
cost 2471674.099999983
th 0.6
sumando k
---------------------------------
k: 5602
count_zero 0
count_one 27000
0.6
cost 2471674.099999983
th 0.6
sumando k
---------------------------------
k: 5603
count_zero 0
count_one 27000
0.6
cost 2471674.099999983
th 0.6
sumando k
---------------------------------
k: 5604
count_zero 0
count_one 27000
0.6
cost 2471674.099999983
th 0.6
sumando k
---------------------------------
k: 5605
count_zero 0
count_one 27000
0.6
cost 2471674.099999983
th 0.6
sumando k
---------------------------------
k: 5606
count_zero 0
count_one 27000
0.6
cost 2471674.099999983
th 0.6
sumando k
---------------------------------
k: 5607
count_zero 0
count_one 27000
0.6
cost 2471674.099999983
th 0.6
sumando k
---------------------------------
k: 5608
count_zero 0
count_one 27000
0.6
cost 2471674.099999983
th 0.6
sumando k

0.6
cost 2471674.099999983
th 0.6
sumando k
---------------------------------
k: 5672
count_zero 0
count_one 27000
0.6
cost 2471674.099999983
th 0.6
sumando k
---------------------------------
k: 5673
count_zero 0
count_one 27000
0.6
cost 2471674.099999983
th 0.6
sumando k
---------------------------------
k: 5674
count_zero 0
count_one 27000
0.6
cost 2471674.099999983
th 0.6
sumando k
---------------------------------
k: 5675
count_zero 0
count_one 27000
0.6
cost 2471674.099999983
th 0.6
sumando k
---------------------------------
k: 5676
count_zero 0
count_one 27000
0.6
cost 2471674.099999983
th 0.6
sumando k
---------------------------------
k: 5677
count_zero 0
count_one 27000
0.6
cost 2471674.099999983
th 0.6
sumando k
---------------------------------
k: 5678
count_zero 0
count_one 27000
0.6
cost 2471674.099999983
th 0.6
sumando k
---------------------------------
k: 5679
count_zero 0
count_one 27000
0.6
cost 2471674.099999983
th 0.6
sumando k
---------------------------------
k:

KeyboardInterrupt: 

In [80]:
[proba[1] for proba in ngb_clf.predict_proba(X_test)]

[0.4870929739154075,
 0.03201905518742649,
 0.36165431926524816,
 0.7459990304309252,
 0.16037009038336214,
 0.3005755294126598,
 0.8081239287132322,
 0.7777166492754485,
 0.266316215021689,
 0.12416326192153469,
 0.1928853831484998,
 0.8635745510467775,
 0.9058707443783496,
 0.7568501335709502,
 0.02923186793394652,
 0.0942865306082414,
 0.6940264991687675,
 0.15942100893957656,
 0.41584068293686327,
 0.7572444190728764,
 0.48422616750970704,
 0.5510115343614391,
 0.44545573664057586,
 0.3772893873072769,
 0.7396798269691341,
 0.6073435728013171,
 0.25359932951759795,
 0.9431732790012803,
 0.3819411402459582,
 0.23693999903524093,
 0.736042518775163,
 0.4042733032782023,
 0.037076170791740665,
 0.40214787701334426,
 0.6718176245802564,
 0.4417406270601709,
 0.6244696214804852,
 0.5763082519545772,
 0.21860990324737667,
 0.3098857171674829,
 0.30667727421587526,
 0.6940264991687675,
 0.3686666206801773,
 0.1777894504851617,
 0.6244696214804852,
 0.14626788649539368,
 0.1071300432897205

In [93]:
df_collector['final_cost'] = df_collector['cost'] - df_collector['learning_cost']

In [107]:
df_collector[df_collector['final_cost'] >0].query("alpha==999725")

,alpha,hyperparams,estimators,learning_rate,max_depth,threshold,cost,learning_cost,train_auc,test_auc,final_cost
3272,999725,NGBClassifier(Base=DecisionTreeRegressor(ccp_a...,50,0.01,"DecisionTreeRegressor(ccp_alpha=0.0, criterion...",0.8,2.47167e+06,0,0.826416,0.824224,2.47167e+06


In [112]:
df_collector[df_collector['final_cost'] >0].describe()

,alpha,hyperparams,estimators,learning_rate,max_depth,threshold,cost,learning_cost,train_auc,test_auc,final_cost
count,837,837,837,837.00,837,837.0,837.0,837.0,837.000000,837.000000,837.0
unique,835,8,1,3.00,4,7.0,12.0,226.0,8.000000,8.000000,226.0
top,386018,NGBClassifier(Base=DecisionTreeRegressor(ccp_a...,50,0.01,"DecisionTreeRegressor(ccp_alpha=0.0, criterion...",0.9,2471674.1,0.0,0.826416,0.824224,2471674.1
freq,2,401,837,711.00,520,316.0,612.0,612.0,401.000000,401.000000,612.0


In [102]:
df_collector[df_collector['final_cost'] >0].min().alpha

10079.0

In [86]:
df_collector.sort_values("cost").iloc[3051]

alpha                                                              596272
hyperparams             NGBClassifier(Base=DecisionTreeRegressor(ccp_a...
estimators                                                             90
learning_rate                                                         0.1
max_depth               DecisionTreeRegressor(ccp_alpha=0.0, criterion...
threshold                                                            0.75
cost                                                          5.70545e+06
cost_common                                                    7.6209e+06
learning_cost                                                 4.20081e+08
learning_cost_common                                          9.54968e+08
train_auc                                                         0.94028
test_auc                                                         0.762959
Name: 1952, dtype: object

In [88]:
df_collector.sort_values("cost").iloc[3051].cost - df_collector.sort_values("cost").iloc[3051].learning_cost

-414375074.8010962